In [ ]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"


class display(object):
    """Display HTML representation of multiple objects"""

    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""

    def __init__(self, *args):
        self.args = args

    def _repr_html_(self):
        return "\n".join(
            self.template.format(a, eval(a)._repr_html_()) for a in self.args
        )

    def __repr__(self):
        return "\n\n".join(a + "\n" + repr(eval(a)) for a in self.args)

In [ ]:
import pandas as pd

pd.__file__

In [ ]:
import numpy as np

a = np.array([[1, 2, 3], [4, 5, 6]])
a

b = a.flatten()
b
c = a.flatten("F")
c

In [ ]:
import pandas as pd

pd.__file__

In [ ]:
from eos.data_io.pool.parquet_pool import ParquetPool
from eos.data_io.pool.avro_pool import AvroPool
from eos.data_io.pool.dask_pool import DaskPool
from eos.data_io.pool.mongo_pool import MongoPool

In [ ]:
from eos.data_io.buffer.mongo_buffer import MongoBuffer

In [ ]:
from eos.data_io.buffer.dask_buffer import DaskBuffer

In [ ]:
# from dataclasses import dataclass, field
# from typing import Optional
# import pandas as pd
# import numpy as np
# from ordered_set import OrderedSet
from eos.data_io.config import drivers, trucks_by_id, drivers_by_id

# DriverSet = OrderedSet(
#     [
#         'wang-cheng',
#         'li-changlong',
#         'chen-hongmei',
#         'zheng-longfei',
#     ]
# )
# DriverSet[0]
# DriverSet.get_loc('wang-cheng')
# DriverSet

In [ ]:
from eos.data_io.eos_struct import StateSpecsCloud, StateSpecsECU

In [ ]:
from eos.data_io.eos_struct.eos_data import StateSpecs

state_specs = StateSpecs()

In [ ]:
state_specs

In [ ]:
from dataclasses import asdict
from eos.data_io.eos_struct.eos_data import (
    StateSpecsCloud,
    StateSpecsECU,
    CloudMixin,
    ECUMixin,
)
from eos.data_io.eos_struct.eos_data import ActionSpecs, RewardSpecs

state_specs = StateSpecsCloud(cloud_interface=CloudMixin())
print(asdict(state_specs))

action_specs = ActionSpecs()
reward_specs = RewardSpecs()
# state_specs = StateSpecsECU(ecu_interface=ECUMixin())
# print(asdict(state_specs))

In [ ]:
from eos.data_io.eos_struct.eos_data import ObservationMetaCloud, ObservationMetaECU

# observation_meta = ObservationMetaCloud(state_specs=StateSpecsCloud(cloud_interface=CloudMixin()))
# print(asdict(observation_meta))
observation_meta = ObservationMetaCloud(
    state_specs=state_specs,
    action_specs=action_specs,
    reward_specs=reward_specs,
    site="Anting",
)
type(observation_meta)
observation_meta.model_dump()

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

ts = pd.to_datetime(datetime.now())
ts_ind = ts + pd.to_timedelta(np.arange(2), "H")
ts_ind
df = pd.DataFrame(a, index=ts_ind, columns=["c1", "c2", "c3"])
df
df.index[0]

In [ ]:
ts
ts_end = pd.to_datetime(datetime.now())
ts_end
span = ts_end - ts
span
span_each_row = span / 4
span_each_row
np.linspace(0, 4, 5)
ts_ser = ts + pd.to_timedelta(np.linspace(1, 4, 4) * span_each_row, unit="s")
ts_ser

In [ ]:
df_ts = pd.DataFrame(np.array([ts]), columns=["timestamp"])
df_ts

In [ ]:
s = np.arange(10)
s

### Generate state

In [ ]:
ss = np.arange(12)
a1 = ss[:4].tolist()
a2 = ss[4:8].tolist()
a3 = ss[8:].tolist()
ss = [a1, a2, a3]
ss
df_s = pd.DataFrame(ss, columns=["t", "v", "p", "b"])  # .set_index('t')
df_s

In [ ]:
np.arange(0, 4 * 20, 20)

In [ ]:
ts_ind = ts + pd.to_timedelta(np.arange(0, 4 * 20, 20), "ms")
ts_ind
ss = np.arange(12)
a1 = ss[:4]
a2 = ss[4:8]
a3 = ss[8:]
df_ss = pd.DataFrame(
    {"timestep": ts_ind, "velocity": a1, "thrust": a2, "brake": a3}
)  # .set_index('timestep')
df_ss.columns.name = "qtuple"
df_ss
df_ss[["velocity", "thrust", "brake"]]

In [ ]:
npa_ss_flatten = df_ss[["velocity", "thrust", "brake"]].to_numpy().flatten()
npa_ss_flatten

In [ ]:
ui_t = df_ss.loc[:, ["velocity", "thrust"]]
ui_t
power_t = ui_t.prod(axis=1)
power_t
power_t.sum()

In [ ]:
state = df_ss.stack().swaplevel(0, 1)
state.name = "state"
state.index.names = ["rows", "idx"]
state.sort_index(inplace=True)
state

In [ ]:
state[["velocity", "thrust", "brake"]]

In [ ]:
# from functools import reduce
#
#
# velocity = pd.Series(ss[:4], name='velocity')
# thrust = pd.Series(ss[4:8], name='thrust')
# brake = pd.Series(ss[8:], name='brake')
# series = [velocity, thrust, brake]
# state = reduce(
#     lambda x, y: pd.merge(x, y, how='outer', left_index=True, right_index=True), series
# )
# state = state.stack().swaplevel(0, 1).sort_index()
# state.name = 'state'
# state
# len(state)

In [ ]:
# from functools import reduce
#
#
# velocity = pd.Series(s[:4], name='velocity')
# thrust = pd.Series(s[4:7], name='thrust')
# brake = pd.Series(s[7:], name='brake')
# series = [velocity, thrust, brake]
# state = reduce(
#     lambda x, y: pd.merge(x, y, how='outer', left_index=True, right_index=True), series
# )
# state = state.stack().swaplevel(0, 1).sort_index()
# state.name = 'state'
# state
# len(state)
#
#

### generate action

In [ ]:
from functools import reduce

a = len(ss) + np.arange(15)
speed_ser = pd.Series(np.linspace(40, 60, 3), name="speed")
# row_ser
row_dict = {f"r{i}": a[i * 5 : i * 5 + 5] for i in np.arange(3)}
row_dict
row_array = a.reshape(3, 5).transpose()
row_array
rows_df = pd.DataFrame(row_array)
rows_df.columns = [f"r{i}" for i in np.arange(3)]
rows_df

action_row_names = [f"r{i}" for i in np.arange(3)]
action_row_names

In [ ]:
ts_ind = ts + pd.to_timedelta(np.arange(5 * 20, 8 * 20, 20), "ms")
ts_ser = pd.Series(ts_ind, name="timestep")
throttle_ser = pd.Series(np.linspace(0, 1.0, 5), name="throttle")
# throttle_ser
dfs = [rows_df, ts_ser, speed_ser, throttle_ser]
action = (
    reduce(
        lambda left, right: pd.merge(
            left, right, how="outer", left_index=True, right_index=True
        ),
        dfs,
    )
    .stack()
    .swaplevel(0, 1)
    .sort_index()
)

action.name = "action"
action.index.names = ["rows", "idx"]
action.index
action
# action = pd.concat([df.DataFrame(row_dict),df.Series(row_ind),df.Series(ts_ind),df.Series(pedal_ind)],axis=1)
# action
# pedal_ind
# action_ind = pd.MultiIndex.from_product([row_ind, ts_ind], names=['row', 'timestamp'])
# a
# speed_row, row_name = *row_dict
# action = (
#     pd.DataFrame(
#         {
#             **row_dict,
#             'throttle': pedal_ind,
#             'speed': [r[1] for r in row_ind],
#         }
#     )  # .set_index('pedal')
#     # .stack()
#     # .swaplevel(0, 1)
#     .sort_index()
# )
# action
# action.columns
# action.columns = pd.MultiIndex.from_arrays(
#     [action.columns, ts_ind], names=['nominalspeed', 'timestep']
# )
# action = action.stack([0, 1]).swaplevel(0, 1).swaplevel(1, 2).sort_index()
# action.name = 'action'
# # action.index.names = ['nominalspeed', 'timestamp']
# # action.index
# action

In [ ]:
action[["r0", "r1", "r2"]]

In [ ]:
# a = len(state) + np.arange(6)
# ar0 = pd.Series(a[:2], name='row0')
# ar1 = pd.Series(a[2:4], name='row1')
# ar2 = pd.Series(a[4:], name='row2')
# series = [ar0, ar1, ar2]
# action = reduce(
#     lambda x, y: pd.merge(x, y, how='outer', left_index=True, right_index=True), series
# )
# action = action.stack().swaplevel(0, 1).sort_index()
# action.name = 'action'
# action
# len(action) + len(state)

In [ ]:
reward = (
    pd.DataFrame({"work": len(ss) + len(a), "timestep": ts_ind[0]}, index=[0])
    .stack()
    .swaplevel(0, 1)
    .sort_index()
)
# reward_index = (reward.name,  ts_ind[0], 0)
reward.index.names = ["rows", "idx"]
reward.name = "reward"
reward.index
reward
# reward.name = 'reward'

In [ ]:
reward["work"][0]
reward.loc[("work", 0)]

In [ ]:
# velocity = pd.Series(len(state) + len(action) + len(reward) + s[:4], name='velocity')
# thrust = pd.Series(len(state) + len(action) + len(reward) + s[4:7], name='thrust')
# brake = pd.Series(len(state) + len(action) + len(reward) + s[7:], name='brake')
# series = [velocity, thrust, brake]
# nstate = reduce(
#     lambda x, y: pd.merge(x, y, how='outer', left_index=True, right_index=True), series
# )
# nstate = nstate.stack().swaplevel(0, 1).sort_index()
# nstate.name = 'nstate'
# nstate
# len(nstate)
ts_ind = ts + pd.to_timedelta(5, "s") + pd.to_timedelta(np.arange(0, 4 * 20, 20), "ms")
ts_ind
ss = np.arange(12) + len(ss) + len(a) + len(reward)
a1 = ss[:4]
a2 = ss[4:8]
a3 = ss[8:]
nstate = (
    pd.DataFrame({"timestep": ts_ind, "velocity": a1, "thrust": a2, "brake": a3})
    # .set_index('timestamp')
    .stack()
    .swaplevel(0, 1)
    .sort_index()
)
nstate.name = "nstate"
nstate.index.names = ["rows", "idx"]
len(nstate)
nstate

In [ ]:
timestamp = pd.Series([ts], name="timestamp")
timestamp.index = pd.MultiIndex.from_product(
    [timestamp.index, [0]], names=["rows", "idx"]
)
timestamp.index
state.index
action.index
reward.index
nstate.index

In [ ]:
timestamp_index = (timestamp.name, "", 0)
# timestamp_index
state_index = [(state.name, *i) for i in state.index]
# state_index
reward_index = [(reward.name, *i) for i in reward.index]
# reward_index
action_index = [(action.name, *i) for i in action.index]
# action_index
nstate_index = [(nstate.name, *i) for i in nstate.index]
# nstate_index

multiindex = pd.MultiIndex.from_tuples(
    [timestamp_index, *state_index, *action_index, *reward_index, *nstate_index]
)
multiindex

In [ ]:
timestamp.index
timestamp.values
timestamp

In [ ]:
observation_list = [timestamp, state, action, reward, nstate]
observation = pd.concat(observation_list)
observation.index = multiindex
observation
# observation.dtype

In [ ]:
idx = pd.IndexSlice
# observation_sorted = observation.sort_index()
actions = (
    observation.sort_index()
    .loc[idx["action", action_row_names, :]]
    .values.astype(np.float32)
)
actions
actions.dtype

In [ ]:
import tensorflow as tf

# actions = tf.convert_to_tensor(actions, dtype=tf.float32)
actions
last_actions = tf.expand_dims(tf.expand_dims(actions, axis=0), axis=0)
last_actions

In [ ]:
states = (
    observation.sort_index()
    .loc[idx["state", ["velocity", "thrust", "brake"], :]]
    .values.astype(np.float32)
)
states = tf.expand_dims(
    tf.expand_dims(tf.convert_to_tensor(states, dtype=tf.float32), axis=0), axis=0
)
states

In [ ]:
observation.index

In [ ]:
observation0 = observation.copy()
observation0.loc["timestamp", "", 0] = ts + pd.Timedelta(1, "h")
observation1 = observation.copy()
observation1.loc["timestamp", "", 0] = ts + pd.Timedelta(2, "h")
observation2 = observation.copy()
observation2.loc["timestamp", "", 0] = ts + pd.Timedelta(3, "h")
observation3 = observation.copy()
observation3.loc["timestamp", "", 0] = ts + pd.Timedelta(4, "h")
observation4 = observation.copy()
observation4.loc["timestamp", "", 0] = ts + pd.Timedelta(5, "h")
observation_list = [
    observation0,
    observation1,
    observation2,
    observation3,
    observation4,
]

In [ ]:
dfs_epi = pd.concat(observation_list, axis=1).transpose()
dfs_epi.columns.names = ["qtuple", "rows", "idx"]
dfs_epi.columns
dfs_epi

In [ ]:
dfs_episode1 = dfs_epi.copy().sort_index(axis=1)
dfs_episode1.set_index(("timestamp", "", 0), inplace=True)
dfs_episode1.index.name = "timestamp"
idx = pd.IndexSlice
dfs_episode1

In [ ]:
import pandas as pd

# Create a sample DataFrame with MultiIndex columns
data = {"col1": ["1", "2", "3"], "col2": ["4.5", "5.6", "6.7"]}
df = pd.DataFrame(data)
df
df.index
df.columns
df.dtypes

In [ ]:
# Convert 'col1' to integer and 'col2' to float
df.columns = pd.MultiIndex.from_tuples([("A", "col1"), ("B", "col2")])
df[("A", "col1")] = df[("A", "col1")].astype(int)
df[("B", "col2")] = df[("B", "col2")].astype(float)

# Check the updated data types
print(df.dtypes)

In [ ]:
df = pd.DataFrame(
    {
        "user1": [2, 4, 21, 21],
        "user2": [6, 13, 76, 76],
        "param1": [0, 2, 0, 1],
        "param2": ["x", "a", "a", "d"],
        "count": [1, 3, 2, 1],
    },
    columns=["user1", "user2", "param1", "param2", "count"],
)
df
df = df.set_index(["user1", "user2", "param1", "param2"])
df
df = df.unstack([2, 3]).sort_index(axis=1)
df
df2 = pd.DataFrame({"user1": [2, 5, 21], "user2": [6, 18, 76]})
df2.columns = pd.MultiIndex.from_product([df2.columns, [""], [""]])
df2
final_df = df2.merge(df, on=["user1", "user2"], how="outer").fillna(0)
final_df
final_df.dtypes

In [ ]:
final_df.loc[:, idx["count", [0, 1, 2], ["a", "x", "d", "a"]]].astype(int)
final_df.loc[:, idx["count", [0, 1, 2], ["a", "x", "d", "a"]]] = final_df.loc[
    :, idx["count", [0, 1, 2], ["a", "x", "d", "a"]]
].astype(int)
final_df
final_df.dtypes
final_df1 = final_df.copy()
final_df1.loc[:, idx["count", [0, 1, 2], ["a", "x", "d", "a"]]] = "a"
final_df1
final_df1.dtypes

In [ ]:
dfs_actions = dfs_episode1.loc[:, idx["action", :, :]]
dfs_actions.dtypes
dfs_actions.columns

In [ ]:
# dfs_actions.loc[:, idx['action', ['r0','r1','r2'],0:5]] = dfs_actions.loc[
#                 :, idx['action', ['r0','r1','r2'],0:5]].astype(int)
# dfs_actions.loc[:, idx['action', ['r0','r1','r2'],0:5]].astype(int, copy=False)
# dfs_actions.dtypes

dfs_actions[("action", "r0", 0)].astype(int)
# dfs_actions.loc[:, ('action', 'r0', 0)]
dfs_actions.loc[:, ("action", "r0", 0)] = dfs_actions[("action", "r0", 0)].astype(int)
dfs_actions[("action", "r0", 0)]

In [ ]:
dfs_episode2 = dfs_episode1.copy()
dfs_episode2
dfs_episode2.dtypes

In [ ]:
dfs_actions = dfs_episode2.loc[
    :, idx["action", ["r0", "r1", "r2", "speed", "throttle"], :]
].astype(float)
dfs_actions
dfs_actions.dtypes

In [ ]:
dfs_episode2.loc[
    :, idx["action", ["r0", "r1", "r2", "speed", "throttle"], :]
] = dfs_actions
dfs_episode2
dfs_episode2.dtypes
dfs_actions1 = dfs_episode2.loc[
    :, idx["action", ["r0", "r1", "r2", "speed", "throttle"], :]
]
dfs_actions1
dfs_actions1.dtypes

In [ ]:
dfs_actions
dfs_actions.dtypes

In [ ]:
dfs_episode2.loc[:, idx["action", "r0", :]] = dfs_episode2.loc[
    :, idx["action", "r0", :]
].astype(int)
dfs_episode2.dtypes
dfs_episode2

In [ ]:
dfs_episode1.loc[
    :, idx["action", ["r0", "r1", "r2", "speed", "throttle"], :]
] = dfs_episode1.loc[
    :, idx["action", ["r0", "r1", "r2", "speed", "throttle"], :]
].astype(
    "float"
)
dfs_episode1.loc[
    :, idx["state", ["brake", "thrust", "velocity"], :]
] = dfs_episode1.loc[:, idx["state", ["brake", "thrust", "velocity"], :]].astype(
    "float"
)
dfs_episode1.loc[
    :, idx["nstate", ["brake", "thrust", "velocity"], :]
] = dfs_episode1.loc[:, idx["nstate", ["brake", "thrust", "velocity"], :]].astype(
    "float"
)
dfs_episode1
dfs_episode1.dtypes
dfs_episode1.columns

### Convert columns types to float

In [ ]:
dfs_epi
dfs_episode = dfs_epi.copy()
dfs_episode.index
dfs_episode.set_index(("timestamp", "", 0), inplace=True)
dfs_episode.sort_index(axis=1, inplace=True)
dfs_episode.index
dfs_episode
dfs_episode.dtypes

In [ ]:
dfs_episode.index.name = "timestamp"
idx = pd.IndexSlice

# # dfs_episode = dfs_episode.astype({idx['state', 'velocity', :]: 'int'})  # Error unhashable type: 'slice' for dictionary
# # sorted_idx = **(idx['state', :, :]:'int')
# # sorted_idx
# dfs_episode.loc[:, idx['state', ['brake', 'thrust', 'velocity'], :]] = dfs_episode.loc[
#     :, idx['state', ['brake', 'thrust', 'velocity'], :]
# ].astype(
#     'float'
# )  # float16 not allowed in parquet
#
# dfs_episode.loc[:, idx['action', ['r0', 'r1', 'r2'], :]] = dfs_episode.loc[
#     :, idx['action', ['r0', 'r1', 'r2'], :]
# ].astype(
#     'float'
# )  # float16 not allowed in parquet
# # dfs_episode['action'] = dfs_episode['action'].astype('float')
# # dfs_episode['reward'] = dfs_episode['reward'].astype('float')
# dfs_episode.loc[:, idx['reward', 'work', 0]] = dfs_episode.loc[
#     :, idx['reward', 'work', 0]
# ].astype(
#     'float'
# )  # float16 not allowed in parquet
#
# # dfs_episode['nstate'] = dfs_episode['nstate'].astype('float')
# dfs_episode.loc[:, idx['nstate', ['brake', 'thrust', 'velocity'], :]] = dfs_episode.loc[
#     :, idx['nstate', ['brake', 'thrust', 'velocity'], :]
# ].astype(
#     'float'
# )  # float16 not allowed in parquet
state_cols_float = [("state", col) for col in ["brake", "thrust", "velocity"]]
action_cols_float = [("action", col) for col in ["r0", "r1", "r2", "speed", "throttle"]]
reward_cols_float = [("reward", "work")]
nstate_cols_float = [("nstate", col) for col in ["brake", "thrust", "velocity"]]
for col in action_cols_float + state_cols_float + reward_cols_float + nstate_cols_float:
    dfs_episode[col[0], col[1]] = dfs_episode[col[0], col[1]].astype(
        "float"
    )  # float16 not allowed in parquet
# dfs_episode['action','timestep'] = dfs_episode['action','timestep'].astype(
#     'datetime64[ns]'
# )  # float16 not allowed in parquet
# dfs_episode['state','timestep'] = dfs_episode['state', 'timestep'].astype(
#     'datetime64[ns]'
# )  # float16 not allowed in parquet
# dfs_episode['nstate','timestep'] = dfs_episode['nstate', 'timestep'].astype(
#     'datetime64[ns]'
# )  # float16 not allowed in parquet
dfs_episode
dfs_episode.dtypes
# dfs_episode.columns
# dfs_epi
# dfs_episode

## prepend two levels of index "vehicle" and "driver" to the DataFrame object

In [ ]:
dfs_episode = pd.concat(
    [dfs_episode], keys=[drivers_by_id["wang-cheng"].pid], names=["driver"]
)
dfs_episode = pd.concat(
    [dfs_episode], keys=[trucks_by_id["VB7"].vid], names=["vehicle"]
)

In [ ]:
dfs_episode.index
dfs_episode.index.dtypes
dfs_episode.columns

In [ ]:
# dfs_episode.columns.set_names(
#     [
#         'qtuple',
#         'rows',
#         'idx',
#     ],
#     level=[0, 1, 2],
#     inplace=True,
# )
dfs_episode
dfs_episode.dtypes

In [ ]:
# dfs_episode['state', 'thrust'] = dfs_episode['state', 'thrust'].astype('float')
# # dfs_episode.loc[:, idx['state', ['brake', 'thrust', 'velocity'], :]] = dfs_episode.loc[
# #     :, idx['state', ['brake', 'thrust', 'velocity'], :]
# # ].astype(
# #     'float'
# # )  # float16 not allowed in parquet
# dfs_episode.loc[:, ('state', ['brake', 'thrust', 'velocity'])] = dfs_episode.loc[
#     :, ('state', ['brake', 'thrust', 'velocity'])
# ].astype(
#     'float'
# )  # float16 not allowed in parquet
# dfs_episode['action', 'r0'] = dfs_episode['action', 'r0'].astype(
#     'float'
# )  # float16 not allowed in parquet
#
# dfs_episode.dtypes

In [ ]:
# idx = pd.IndexSlice
# dfs_episode.loc[:, idx['state', :, :]] = dfs_episode.loc[:, idx['state', :, :]].astype(
#     'int'
# )
# dfs_episode.loc[:, idx['action', :, :]] = dfs_episode.loc[
#     :, idx['action', :, :]
# ].astype('float16')
# dfs_episode.loc[:, idx['reward', :, :]] = dfs_episode.loc[
#     :, idx['reward', :, :]
# ].astype('float16')
# dfs_episode.loc[:, idx['nstate', :, :]] = dfs_episode.loc[
#     :, idx['nstate', :, :]
# ].astype('float16')
#
# vel_1 = dfs_episode[[('state', 'velocity', 1)]]
# vel_1.dtypes
# vel_1.index
# vel_1.values
# # type(vel_1)
# vel_1.iloc[0]
# type(vel_1.iloc[0])
# type(vel_1.iloc[0].values[0])

## Add a level of index for episode

In [ ]:
episodestart = ts - pd.Timedelta(1, "h")
dfs_episode = pd.concat([dfs_episode], keys=[ts], names=["episodestart"])
dfs_episode = dfs_episode.swaplevel(1, 0, axis=0)
dfs_episode = dfs_episode.swaplevel(1, 2, axis=0)
dfs_episode.sort_index(inplace=True)
dfs_episode.index
dfs_episode.columns
dfs_episode

In [ ]:
dfs_episode.dtypes

In [ ]:
vb7 = trucks_by_id["VB7"]

### Encoding MultiIndexed DataFrame to documents array of MongoDB record type (nested dictionary)

In [ ]:
from eos.data_io.eos_struct import (
    ObservationMetaCloud,
    StateSpecs,
    StateSpecsCloud,
    StateUnitCodes,
    ActionSpecs,
    RewardSpecs,
)

state_specs_cloud = StateSpecsCloud()
observation_meta = ObservationMetaCloud(
    state_specs=state_specs_cloud,
    action_specs=action_specs,
    reward_specs=reward_specs,
    site="Anting",
)
#     state_specs=StateSpecs(
#         interface='cloud',
#         state_unit_codes=StateUnitCodes(
#             state_unit_code='kph',
#             thrust_unit_code='pct',
#             brake_unit_code='pct',
#         ),
#         state_number =3,
#         unit_number_per_state = vb7.state_number_per_state,
#     ),
#     action_specs=ActionSpecs(
#         action_unit_code='nm',
#         action_row_number=trucks_by_id['VB7'].torque_table_row_num_flash,
#         action_column_number=len(trucks_by_id['VB7'].pedal_scale),
#     ),
#     reward_specs=RewardSpecs(
#         reward_unit_code='wh',
#         reward_number=1
#     ),
#     site=trucks_by_id['VB7'].site,
# )
observation_meta
observation_meta.model_dump()

In [ ]:
dfs_episode
#
# records = dfs_episode.to_dict('records')
# # records
# record_nested = [[{key: value} for key, value in record.items()] for record in records]
# record_nested
# ser = dfs_episode.to_dict('split')
# ser

In [ ]:
def nest(d: dict) -> dict:
    result = {}
    for key, value in d.items():
        target = result
        for k in key[:-1]:
            target = target.setdefault(k, {})
        target[key[-1]] = value
    return result


def df_to_nested_dict(df_multi_indexed_col: pd.DataFrame) -> dict:
    d = df_multi_indexed_col.to_dict("index")
    return {k: nest(v) for k, v in d.items()}


def eos_df_to_nested_dict(episode: pd.DataFrame) -> dict:
    """
    Convert a eos dataframe with multi-indexed columns to a nested dictionary
    Remove all the levels of the multi-indexed columns except for 'timestamp'
    Keep only the timestamp as the single key for the nested dictionary
    """
    dict_nested = df_to_nested_dict(
        episode
    )  # for multi-indexed dataframe, the index in the first level of the dictionary is still a tuple!
    indices_dict = [
        {episode.index.names[i]: level for i, level in enumerate(levels)}
        for levels in episode.index
    ]  # all elements in the array should have the same vehicle, driver, episodestart
    single_key_dict = {
        idx["timestamp"]: dict_nested[key]
        for idx, key in zip(indices_dict, dict_nested)
    }

    return single_key_dict

In [ ]:
dfs_episode.index

In [ ]:
dict_nested = df_to_nested_dict(dfs_episode)
dict_nested
indices_dict = [
    {dfs_episode.index.names[i]: level for i, level in enumerate(levels)}
    for levels in dfs_episode.index
]  # all elements in the array should have the same vehicle, driver, episodestart

In [ ]:
# dict_nested = [{i: dfs_episode.xs(i).to_dict('index') for i in level} for level in dfs_episode.index.levels]
# dfs_index = dfs_episode.to_dict('index')
# dfs_index


# dfs_episode
# dfs_episode.columns

# dict1 = {key: nest(value) for key, value in dfs_episode.to_dict('index').items()}
# dict1

dict_nested = eos_df_to_nested_dict(dfs_episode)
dict_nested
# dict_nested = {level:
#                    {i: dfs_episode.xs(i).to_dict('index') for i in level }
#                for level in dfs_episode.index.levels}
# dict_nested = {level: dfs_episode.xs(level).to_dict('index')
#                for level in dfs_episode.columns.levels[0]}
# dict_nested
len(dict_nested)

In [ ]:
# dict_df_xs = {level: dfs_episode.xs(level) for level in dfs_episode.index.levels[0]}
# dict_df_xs['VB7']
len(dict_nested)
dfs_episode
dict_nested

In [ ]:
indices = dfs_episode.index
indices.name
indices.names
indices
indices_dict = [
    {indices.names[i]: level for i, level in enumerate(levels)} for levels in indices
]
indices_dict
len(indices_dict)

In [ ]:
single_key_dict = [
    (idx["timestamp"], dict_nested[key]) for idx, key in zip(indices_dict, dict_nested)
]
single_key_dict[3]
len(single_key_dict)

### Decode episode into one document of mongo episode type (nested dictionaries)


In [ ]:
from dataclasses import asdict

episode_meta = indices_dict[0].copy()
try:
    episode_meta.pop("timestamp")
except KeyError:
    print(f"Key 'timestamp' not found")

episode_meta["seq_len"] = len(single_key_dict)
print(episode_meta)
meta = {**(observation_meta.dict()), **episode_meta}
meta

In [ ]:
single_key_dict = {
    idx["timestamp"]: dict_nested[key] for idx, key in zip(indices_dict, dict_nested)
}

In [ ]:
from eos.data_io.eos_struct import DataFrameDoc

doc = DataFrameDoc(
    timestamp=meta["episodestart"],
    meta=meta,
    observation=single_key_dict,
)
doc

In [ ]:
indices_dict[0]

In [ ]:
# from dataclasses import asdict
#
# rows_multi = [
#     {
#         'timestamp': idx['timestamp'],
#         'meta': {**idx, **(asdict(meta))},
#         'observation': {key: value for key, value in record.items()},
#     }
#     for (idx, record) in zip(indices_dict, records)
# ]
# rows_multi

### Encode the records of an episode into rows of nested dictionaries

In [ ]:
from dataclasses import asdict

rows = [
    {
        "timestamp": idx["timestamp"],
        "meta": {**idx, **(observation_meta.dict())},
        "observation": dict_nested[key],
    }
    for (idx, key) in zip(indices_dict, dict_nested)
]
rows
len(rows)

## Decoding MongoDB dict to MultiIndexed DataFrame

In [ ]:
df_rows = pd.DataFrame(rows)
df_rows
df_rows["observation"][0]

In [ ]:
dict_observations = [
    {key: value for key, value in row.items()} for row in df_rows["observation"]
]
dict_observations

### Decoding MongoDB dict to MultiIndexed DataFrame
first get the multiindex as tuple from nested keys

In [ ]:
dict_observations = {
    (
        meta["episodestart"],
        meta["vehicle"],
        meta["driver"],
        meta["timestamp"],
        key1,
        key2,
        key3,
    ): value
    for meta, obs in zip(df_rows["meta"], df_rows["observation"])
    for key1, obs1 in obs.items()
    for key2, obs2 in obs1.items()
    for key3, value in obs2.items()
}

dict_observations
len(dict_observations)

In [ ]:
ser_decoded = pd.Series(dict_observations)
ser_decoded.index.names = [
    "episodestart",
    "vehicle",
    "driver",
    "timestamp",
    "qtuple",
    "rows",
    "idx",
]
ser_decoded

In [ ]:
df_decoded = ser_decoded.unstack(level=["qtuple", "rows", "idx"])
df_decoded
df_decoded.columns
# df_decoded.reset_index()

In [ ]:
# df_decoded.reset_index()
# df_decoded

In [ ]:
# ser_action = ser_decoded.loc[('action', ['r0', 'r1', 'r2', 'throttle'])]
# df_action = ser_action.unstack(level=[4, 5])
idx = pd.IndexSlice
df_action = df_decoded.loc[:, idx["action", ["r0", "r1", "r2", "throttle"]]]
df_action
# df_action.set_index(('action', 'throttle'))5369@

### Decoding MongoDB dict to MultiIndexed fully described DataFrame

In [ ]:
dict_observations_list = [
    {
        (
            meta["episodestart"],
            meta["vehicle"],
            meta["driver"],
            meta["timestamp"],
            key1,
            key2,
            key3,
        ): value
        for key1, obs1 in obs.items()
        for key2, obs2 in obs1.items()
        for key3, value in obs2.items()
    }
    for meta, obs in zip(df_rows["meta"], df_rows["observation"])
]

dict_observations_list[0]
len(dict_observations_list)

In [ ]:
idx = pd.IndexSlice
ser_decoded = pd.Series(dict_observations_list[0])
ser_decoded.index.names = [
    "episodestart",
    "vehicle",
    "driver",
    "timestamp",
    "qtuple",
    "rows",
    "idx",
]
# ser_decoded
# ser_decoded.index
ser_action = ser_decoded.loc[idx[:, :, :, :, "action", ["r0", "r1", "r2", "throttle"]]]
# ser_action.index
df_action = ser_action.unstack([0, 1, 2, 3, 4, 5])
# df_action.index
# df_action.columns
multiindex = df_action.columns
# multiindex[0]
multiindex[-1]
# df_action.index.names
# multiindex = (*df_action.index.names,('action', 'throttle'))
# multiindex
df_action.set_index(multiindex[-1], inplace=True)
df_action
df_action.columns
# df_action.set_index(idx[:, :, :, :, 'action', 'throttle'], inplace=True)
# df_action = pd.concat([df_action], keys=[('action','throttle')], names=['qtuple', 'rows']).set
# multiindex = (df_action.index,'action', 'throttle'))
# df_action.set_index(multiindex, inplace=True)
# df_action

In [ ]:
action_timestep = ser_decoded.loc[idx[:, :, :, :, "action", "timestep"]]
action_timestep
action_speed = ser_decoded.loc[idx[:, :, :, :, "action", "speed"]]
action_speed
action_multicol = [
    (*column, speed, timestep)
    for column, timestep, speed in zip(df_action.columns, action_timestep, action_speed)
]
action_multicol

df_action.columns = pd.MultiIndex.from_tuples(
    action_multicol,
    names=[
        "episodestart",
        "vehicle",
        "driver",
        "timestamp",
        "qtuple",
        "rows",
        "speed",
        "timestep",
    ],
)
df_action
df_action.values.flatten()

In [ ]:
ser_state = ser_decoded.loc[
    idx[:, :, :, :, "state", ["brake", "thrust", "velocity", "timestep"]]
]
# ser_state
df_state = ser_state.unstack([0, 1, 2, 3, 4, 5])
# df_state = ser_state.unstack(level=[0, 1]).set_index(('state', 'timestep'))
# df_state
multiindex = df_state.columns
multiindex[-1]
df_state.set_index(multiindex[-1], inplace=True)
# multiindex[idx[:,:,:,:,:,'timestep']]
# df_state.set_index(multiindex[idx[:,:,:,:,:,'timestep']], inplace=True)
df_state
# df_state.values

In [ ]:
idx = pd.IndexSlice
ser_decoded = pd.Series(dict_observations_list[0])
ser_decoded.index.names = [
    "episodestart",
    "vehicle",
    "driver",
    "timestamp",
    "qtuple",
    "rows",
    "idx",
]
ser_action = ser_decoded.loc[idx[:, :, :, :, "action", ["r0", "r1", "r2", "throttle"]]]
df_action = ser_action.unstack([0, 1, 2, 3, 4, 5])
multiindex = df_action.columns
df_action.set_index(multiindex[-1], inplace=True)

action_timestep = ser_decoded.loc[idx[:, :, :, :, "action", "timestep"]]
action_speed = ser_decoded.loc[idx[:, :, :, :, "action", "speed"]]
action_multicol = [
    (*column, speed, timestep)
    for column, timestep, speed in zip(df_action.columns, action_timestep, action_speed)
]
df_action.columns = pd.MultiIndex.from_tuples(
    action_multicol,
    names=[
        "episodestart",
        "vehicle",
        "driver",
        "timestamp",
        "qtuple",
        "rows",
        "speed",
        "timestep",
    ],
)
df_action


ser_state = ser_decoded.loc[
    idx[:, :, :, :, "state", ["brake", "thrust", "velocity", "timestep"]]
]
df_state = ser_state.unstack([0, 1, 2, 3, 4, 5])
multiindex = df_state.columns
df_state.set_index(multiindex[-1], inplace=True)
df_state


ser_nstate = ser_decoded.loc[
    idx[:, :, :, :, "nstate", ["brake", "thrust", "velocity", "timestep"]]
]
df_nstate = ser_nstate.unstack([0, 1, 2, 3, 4, 5])
multiindex = df_nstate.columns
df_nstate.set_index(multiindex[-1], inplace=True)
df_nstate

In [ ]:
ser_reward = ser_decoded.loc[idx[:, :, :, :, "reward", ["work", "timestep"]]]
# ser_reward = pd.Series(
#     data=[ser_reward[('reward', 'work', 0)]],
#     index=[ser_reward[('reward', 'timestep', 0)]],
#     name='work',
# )
ser_reward = ser_reward.unstack([0, 1, 2, 3, 4, 5])
multiindex = ser_reward.columns
ser_reward.set_index(multiindex[-1], inplace=True)
ser_reward

In [ ]:
# ser_reward = ser_decoded.loc[idx[:,:,:,:,'reward', ['work', 'timestep']]]
# ser_reward = pd.Series(
#     data=[ser_reward[multiindex[0]]],
#     index=[ser_reward[multiindex[1]]],
#     name='work',
# )
# ser_reward

In [ ]:
df_actions = []
df_states = []
df_nstates = []
ser_rewards = []

for dict_observations in dict_observations_list:
    idx = pd.IndexSlice
    ser_decoded = pd.Series(dict_observations)
    ser_decoded.index.names = [
        "episodestart",
        "vehicle",
        "driver",
        "timestamp",
        "qtuple",
        "rows",
        "idx",
    ]

    ser_action = ser_decoded.loc[
        idx[:, :, :, :, "action", ["r0", "r1", "r2", "throttle"]]
    ]
    df_action = ser_action.unstack([0, 1, 2, 3, 4, 5])
    multiindex = df_action.columns
    df_action.set_index(multiindex[-1], inplace=True)

    action_timestep = ser_decoded.loc[idx[:, :, :, :, "action", "timestep"]]
    action_speed = ser_decoded.loc[idx[:, :, :, :, "action", "speed"]]
    action_multicol = [
        (*column, speed, timestep)
        for column, timestep, speed in zip(
            df_action.columns, action_timestep, action_speed
        )
    ]
    df_action.columns = pd.MultiIndex.from_tuples(
        action_multicol,
        names=[
            "episodestart",
            "vehicle",
            "driver",
            "timestamp",
            "qtuple",
            "rows",
            "speed",
            "timestep",
        ],
    )
    # df_action
    df_actions.append(df_action)

    ser_state = ser_decoded.loc[
        idx[:, :, :, :, "state", ["brake", "thrust", "velocity", "timestep"]]
    ]
    df_state = ser_state.unstack([0, 1, 2, 3, 4, 5])
    multiindex = df_state.columns
    df_state.set_index(multiindex[-1], inplace=True)
    # df_state
    df_states.append(df_state)

    ser_nstate = ser_decoded.loc[
        idx[:, :, :, :, "nstate", ["brake", "thrust", "velocity", "timestep"]]
    ]
    df_nstate = ser_nstate.unstack([0, 1, 2, 3, 4, 5])
    multiindex = df_nstate.columns
    df_nstate.set_index(multiindex[-1], inplace=True)
    # df_nstate
    df_nstates.append(df_nstate)

    ser_reward = ser_decoded.loc[idx[:, :, :, :, "reward", ["work", "timestep"]]]
    ser_reward = ser_reward.unstack([0, 1, 2, 3, 4, 5])
    multiindex = ser_reward.columns
    ser_reward.set_index(multiindex[-1], inplace=True)
    # ser_reward
    ser_rewards.append(ser_reward)

### simply concatenate the pandas dataframes leads to confusion
please see the following.


In [ ]:
df_actions_all = pd.concat(df_actions)
df_actions_all

### the following is the correct way to extract columns, index and values from the pandas dataframes

and stack them into numpy arrays for further analysis

In [ ]:
npa_action_columns_list = [df_action.columns for df_action in df_actions]
# npa_action_columns_list
npa_action_indices_list = [df_action.index for df_action in df_actions]
# npa_action_indices_list
npa_state_columns_list = [df_state.columns for df_state in df_states]
npa_state_indices_list = [df_state.index for df_state in df_states]
npa_nstate_columns_list = [df_nstate.columns for df_nstate in df_nstates]
npa_nstate_indices_list = [df_nstate.index for df_nstate in df_nstates]
npa_reward_columns_list = [ser_reward.columns for ser_reward in ser_rewards]
npa_reward_indices_list = [ser_reward.index for ser_reward in ser_rewards]

In [ ]:
npa_action_columns_list[0]
npa_action_indices_list[0]
ts_array = []
for columns in npa_action_columns_list:
    timestep = columns.get_level_values("timestep")
    ts_array.append(timestep)
    # print(timestep)
    # print(npa_action_columns_list[0].loc[idx[:,:,:,timestep,:,:], :])
    # print(npa_action_indices_list[0].loc[idx[:,:,:,timestep,:,:], :])

ts_array = pd.DataFrame(np.stack(ts_array).T)
ts_array
ts_array.describe()
# ts_array.average()

In [ ]:
npa_reward_columns_list[0]
npa_reward_indices_list[0]

In [ ]:
df_states[0]
df_states[0].values.flatten()
df_states[0].loc[
    :, idx[:, :, :, :, "state", ["velocity", "thrust", "brake"]]
]  ## keep the order of input
df_states[0].loc[
    :, idx[:, :, :, :, "state", ["velocity", "thrust", "brake"]]
].values.flatten()
# df_actions[0]
# df_actions[0].values.flatten()

In [ ]:
npa_states_list = [
    df_state.loc[
        :, idx[:, :, :, :, "state", ["velocity", "thrust", "brake"]]
    ].values.T.flatten()
    for df_state in df_states
]  # transpose T is necessary for column-wise stacking and flatten
npa_states = np.stack(npa_states_list).astype("float")
npa_states


npa_actions_list = [
    df_action.loc[:, idx[:, :, :, :, "action", ["r0", "r1", "r2"]]].values.T.flatten()
    for df_action in df_actions
]
npa_actions = np.stack(npa_actions_list).astype("float")
npa_actions

npa_rewards_list = [ser_reward.values.T.flatten() for ser_reward in ser_rewards]
npa_rewards = np.stack(npa_rewards_list).astype("float")
npa_rewards

npa_nstates_list = [
    df_nstate.loc[
        :, idx[:, :, :, :, "nstate", ["velocity", "thrust", "brake"]]
    ].values.T.flatten()
    for df_nstate in df_nstates
]
npa_nstates = np.stack(npa_nstates_list).astype("float")
npa_nstates

# np.concatenate(df_actions.values)

In [ ]:
type(npa_states)
npa_states

In [ ]:
npa_actions

In [ ]:
npa_nstates

In [ ]:
npa_rewards

In [ ]:
# df_action = ser_action.unstack(level=[0, 1]).set_index(('action', 'throttle'))
# action_timestep = ser_decoded.loc[('action', 'timestep')]
# action_speed = ser_decoded.loc[('action', 'speed')]
#
# action_multicol = [
#     (*column, speed, timestep)
#     for column, timestep, speed in zip(df_action.columns, action_timestep, action_speed)
# ]
# df_action.columns = pd.MultiIndex.from_tuples(
#     action_multicol, names=['qtuple', 'rows', 'speed', 'timestep']
# )
# df_action
# df_action.values.flatten()
#
# ser_state = ser_decoded.loc[:, idx['state', ['brake', 'thrust', 'velocity', 'timestep']]]
# # ser_state
# df_state = ser_state.unstack(level=[0, 1]).set_index(('state', 'timestep'))
# df_state
#
# df_state.values
#
# ser_nstate = ser_decoded.loc[:, idx['nstate', ['brake', 'thrust', 'velocity', 'timestep']]]
# # ser_state
# df_nstate = ser_nstate.unstack(level=[0, 1]).set_index(('nstate', 'timestep'))
# df_nstate
#
# ser_reward = ser_decoded.loc[:, idx['reward', ['work', 'timestep']]]
# ser_reward = pd.Series(
#     data=[ser_reward[('reward', 'work', 0)]],
#     index=[ser_reward[('reward', 'timestep', 0)]],
#     name='work',
# )
# ser_reward

In [ ]:
# ser_decoded = pd.Series(dict_observations)
# ser_action = ser_decoded.loc[('action', ['r0', 'r1', 'r2', 'throttle'])]
# df_action = ser_action.unstack(level=[0, 1]).set_index(('action', 'throttle'))
# action_timestep = ser_decoded.loc[('action', 'timestep')]
# action_speed = ser_decoded.loc[('action', 'speed')]
#
# action_multicol = [
#     (*column, speed, timestep)
#     for column, timestep, speed in zip(df_action.columns, action_timestep, action_speed)
# ]
# df_action.columns = pd.MultiIndex.from_tuples(
#     action_multicol, names=['qtuple', 'rows', 'speed', 'timestep']
# )
# df_action
# df_action.values.flatten()
#
# ser_state = ser_decoded.loc[('state', ['brake', 'thrust', 'velocity', 'timestep'])]
# # ser_state
# df_state = ser_state.unstack(level=[0, 1]).set_index(('state', 'timestep'))
# df_state
#
# df_state.values
#
# ser_nstate = ser_decoded.loc[('nstate', ['brake', 'thrust', 'velocity', 'timestep'])]
# # ser_state
# df_nstate = ser_nstate.unstack(level=[0, 1]).set_index(('nstate', 'timestep'))
# df_nstate
#
# ser_reward = ser_decoded.loc[('reward', ['work', 'timestep'])]
# ser_reward = pd.Series(
#     data=[ser_reward[('reward', 'work', 0)]],
#     index=[ser_reward[('reward', 'timestep', 0)]],
#     name='work',
# )
# ser_reward

## Encoding sampled multiple episodes from Mongo episode documents to multiindexed DataFrame

### generate mutliple episodes of mongo episode docs

In [ ]:
dfs_episode

## create epsiode with different lengths

In [ ]:
dfs_episodex = dfs_episode.copy()
dfs_episodex.index = dfs_episodex.index.set_levels([episodestart], level="episodestart")
dfs_episodex.index = dfs_episodex.index.set_levels(
    [[trucks_by_id["VB4"].vid], [drivers_by_id["li-changlong"].pid]],
    level=["vehicle", "driver"],
    verify_integrity=False,
)
dfs_episodex
ts_index = dfs_episodex.index.unique(level="timestamp")

idx_num = len(ts_index)
drop_num = np.random.randint(low=1, high=idx_num - 1)
ts_index_to_drop = np.random.choice(ts_index, drop_num, replace=False)
ts_index_to_drop
dfs_zigzag = dfs_episodex.drop(index=ts_index_to_drop, level="timestamp")
dfs_zigzag

In [ ]:
ts_new = pd.to_datetime((datetime.now()))
ts_new
episodestart = ts_new - pd.Timedelta(2, "d")
episodestart

dfs_episode0 = dfs_episode.copy()
dfs_episode0.index = dfs_episode0.index.set_levels([episodestart], level="episodestart")
dfs_episode0.index = dfs_episode0.index.set_levels(
    [[trucks_by_id["VB7"].vid], [drivers_by_id["zheng-longfei"].pid]],
    level=["vehicle", "driver"],
    verify_integrity=False,
)
ts_index = dfs_episode0.index.unique(level="timestamp")
idx_num = len(ts_index)
drop_num = np.random.randint(low=1, high=idx_num - 1)
ts_index_to_drop = np.random.choice(ts_index, drop_num, replace=False)
dfs_episode0 = dfs_episode0.drop(index=ts_index_to_drop, level="timestamp")

dfs_episode1 = dfs_episode.copy()
dfs_episode1.index = dfs_episode1.index.set_levels(
    [episodestart - pd.Timedelta(3, "d")], level="episodestart"
)
dfs_episode1.index = dfs_episode1.index.set_levels(
    [[trucks_by_id["MP73"].vid], [drivers_by_id["wang-cheng"].pid]],
    level=["vehicle", "driver"],
    verify_integrity=False,
)
ts_index = dfs_episode1.index.unique(level="timestamp")
idx_num = len(ts_index)
drop_num = np.random.randint(low=1, high=idx_num - 1)
ts_index_to_drop = np.random.choice(ts_index, drop_num, replace=False)
dfs_episode1 = dfs_episode1.drop(index=ts_index_to_drop, level="timestamp")

dfs_episode2 = dfs_episode.copy()
dfs_episode2.index = dfs_episode2.index.set_levels(
    [episodestart - pd.Timedelta(4, "d")], level="episodestart"
)
dfs_episode2.index = dfs_episode2.index.set_levels(
    [[trucks_by_id["VB7"].vid], [drivers_by_id["wang-cheng"].pid]],
    level=["vehicle", "driver"],
    verify_integrity=False,
)
ts_index = dfs_episode2.index.unique(level="timestamp")
idx_num = len(ts_index)
drop_num = np.random.randint(low=1, high=idx_num - 1)
ts_index_to_drop = np.random.choice(ts_index, drop_num, replace=False)
dfs_episode2 = dfs_episode2.drop(index=ts_index_to_drop, level="timestamp")

dfs_episode3 = dfs_episode.copy()
dfs_episode3.index = dfs_episode3.index.set_levels(
    [episodestart - pd.Timedelta(5, "d")], level="episodestart"
)
dfs_episode3.index = dfs_episode3.index.set_levels(
    [[trucks_by_id["MP73"].vid], [drivers_by_id["zheng-longfei"].pid]],
    level=["vehicle", "driver"],
    verify_integrity=False,
)

ts_index = dfs_episode3.index.unique(level="timestamp")
idx_num = len(ts_index)
drop_num = np.random.randint(low=1, high=idx_num - 1)
ts_index_to_drop = np.random.choice(ts_index, drop_num, replace=False)
dfs_episode3 = dfs_episode3.drop(index=ts_index_to_drop, level="timestamp")
dfs_episode3

In [ ]:
from functools import reduce

episodes = [dfs_episode, dfs_episode0, dfs_episode1, dfs_episode2, dfs_episode3]
try:
    dfs_episode_all = reduce(
        lambda left, right,: pd.concat([left, right], axis=0), episodes
    )
except Exception as e:
    print(e)
# dfs_episode_all.sort_index(inplace=True)
# dfs_episode_all = dfs_episode_all.swaplevel(1, 0, axis=0)
# dfs_episode_all = dfs_episode_all.swaplevel(1, 2, axis=0)
dfs_episode_all.sort_index(inplace=True)
dfs_episode_all = dfs_episode_all[["state", "action", "reward", "nstate"]]
display("dfs_episode_all")
dfs_episode_all.index
dfs_episode_all.columns

In [ ]:
docs = []
for episode in episodes:
    dict_nested = eos_df_to_nested_dict(episode)
    indices_dict = [
        {indices.names[i]: level for i, level in enumerate(levels)}
        for levels in episode.index
    ]
    single_key_dict = {
        idx["timestamp"]: dict_nested[key]
        for idx, key in zip(indices_dict, dict_nested)
    }

    episode_meta = indices_dict[0].copy()
    try:
        episode_meta.pop("timestamp")
    except KeyError:
        print(f"Key 'timestamp' not found")

    episode_meta["seq_len"] = len(single_key_dict)

    meta = {**(observation_meta.dict()), **episode_meta}

    doc = DataFrameDoc(
        timestamp=meta["episodestart"],
        meta=meta,
        observation=single_key_dict,
    )
    docs.append(doc)

docs

### encode multiple episodes of mongo documents to DataFrames

In [ ]:
df_docs = pd.DataFrame(docs)
df_docs

In [ ]:
dict_observations = [
    {key: value for key, value in row.items()} for row in df_docs["observation"]
]
dict_observations

### decoding observations (episodes)

In [ ]:
dict_observations = {
    (
        meta["vehicle"],
        meta["driver"],
        meta["episodestart"],
        key1,
        key2,
        key3,
        key4,
    ): value
    for meta, obs in zip(df_docs["meta"], df_docs["observation"])
    for key1, obs1 in obs.items()
    for key2, obs2 in obs1.items()
    for key3, obs3 in obs2.items()
    for key4, value in obs3.items()
}

dict_observations
len(dict_observations)

In [ ]:
ser_decoded = pd.Series(dict_observations)
ser_decoded.index.names = [
    "vehicle",
    "driver",
    "episodestart",
    "timestamp",
    "qtuple",
    "rows",
    "idx",
]
ser_decoded
df_decoded = ser_decoded.unstack(level=["qtuple", "rows", "idx"])
df_decoded

In [ ]:
episodestart_index = df_decoded.index.unique(level="episodestart")
# rewards.values
episodestart_index

df_rewards = df_decoded["reward", "work"]
# rewards.index.get_level_values('episodestart')
idx = pd.IndexSlice
np_r_n_t = [
    df_rewards.loc[idx[:, :, ep_start, :]].values.tolist()
    for ep_start in episodestart_index
]
# np_r_n_t = [ rewards.loc[:,:,ep_start].values.tolist()   for ep_start in episodestart_index]
np_r_n_t
np_r_n_t[0]

In [ ]:
df_rewards.loc[:, :, episodestart_index[0]]

In [ ]:
# from keras.utils import pad_sequences
# from tensorflow import keras
from keras.utils import pad_sequences

r_n_t = pad_sequences(np_r_n_t, padding="post", dtype="float32", value=-10000.0)
r_n_t
r_n_t.shape

In [ ]:
idx = pd.IndexSlice
states = df_decoded.loc[:, idx["state", ["velocity", "thrust", "brake"]]]
states

In [ ]:
np_s_n_t = [
    states.loc[idx[:, :, ep_start, :]].values.tolist()
    for ep_start in episodestart_index
]
s_n_t = pad_sequences(np_s_n_t, padding="post", dtype="float32", value=-10000.0)
print(s_n_t)
s_n_t.shape

In [ ]:
torque_table_row_names = ["r0", "r1", "r2"]
actions = df_decoded.loc[:, idx["action", torque_table_row_names]]
np_a_n_t = [
    actions.loc[idx[:, :, ep_start, :]].values.tolist()
    for ep_start in episodestart_index
]
a_n_t = pad_sequences(np_a_n_t, padding="post", dtype="float32", value=-10000.0)
print(a_n_t)
a_n_t.shape

In [ ]:
s_n_t.shape
a_n_t.shape
s_n_t
a_n_t

In [ ]:
idx = pd.IndexSlice
nstates = df_decoded.loc[:, idx["nstate", ["velocity", "thrust", "brake"]]]
print(nstates)
np_ns_n_t = [
    nstates.loc[idx[:, :, ep_start, :]].values.tolist()
    for ep_start in episodestart_index
]
ns_n_t = pad_sequences(np_ns_n_t, padding="post", dtype="float32", value=-10000.0)
print(ns_n_t)
ns_n_t.shape

In [ ]:
s_n_t_ = s_n_t[:, 1:, :]
s_n_t_.shape
s_n_t_
a_n_t_ = a_n_t[:, :-1, :]
a_n_t_.shape
a_n_t_

h_n_t = np.concatenate((s_n_t_, a_n_t_), axis=2)
h_n_t.shape
h_n_t

# split batches into trucated sequences for TBPTT

In [ ]:
obs_n_t = zip(s_n_t, a_n_t, r_n_t, ns_n_t)
type(obs_n_t)

In [ ]:
s_n_t
s_n_t.shape

In [ ]:
split_num = s_n_t.shape[1] // 2 + 1
print(f"split_num: {split_num}")
s_n_t_splits = np.array_split(s_n_t, split_num, axis=1)
len(s_n_t_splits)
s_n_t_splits[0]
s_n_t_splits[1]
s_n_t_splits[2]

In [ ]:
split_num = s_n_t.shape[1] // 2 + 1
print(f"split_num: {split_num}")

for i, batch_t in enumerate(
    zip(
        np.array_split(s_n_t, split_num, axis=1),
        np.array_split(a_n_t, split_num, axis=1),
        np.array_split(r_n_t, split_num, axis=1),
        np.array_split(ns_n_t, split_num, axis=1),
    )
):
    s_n_t_b, a_n_t_b, r_n_t_b, ns_n_t_b = batch_t
    print(f"batch number: {i}")
    print(f"s_n_t_b: {s_n_t_b}")
    print(f"a_n_t_b: {a_n_t_b}")
    print(f"r_n_t_b: {r_n_t_b}")
    print(f"ns_n_t_b: {ns_n_t_b}")

## encoding in batches


In [ ]:
df_rows = pd.DataFrame(docs)
df_rows

In [ ]:
dict_observations = [
    {
        (
            meta["vehicle"],
            meta["driver"],
            meta["episodestart"],
            timestamp,
            qtuple,
            rows,
            idx,
        ): value
        for timestamp, obs1 in obs.items()
        for qtuple, obs2 in obs1.items()
        for rows, obs3 in obs2.items()
        for idx, value in obs3.items()
    }
    for meta, obs in zip(df_rows["meta"], df_rows["observation"])
]

# dict_observations
len(dict_observations)

batch = []
for dict_obs in dict_observations:
    ser_decoded = pd.Series(dict_obs)
    ser_decoded.index.names = [
        "vehicle",
        "driver",
        "episodestart",
        "timestamp",
        "qtuple",
        "rows",
        "idx",
    ]
    df_decoded = ser_decoded.unstack(level=["qtuple", "rows", "idx"])  # type: ignore
    df_decoded.sort_index(inplace=True, axis=1)
    batch.append(df_decoded)  # qtuple, rows, index

# batch.sort_index(inplace=True, axis=0)
# must not sort_index, otherwise the order of the columns will be changed, if there were duplicated episodes
index_names = batch[0].index.names
df_episodes = pd.concat(batch, keys=range(len(batch)), names=["batch"])
df_episodes

In [ ]:
df_episodes.loc[1, :, :, :, :]["reward", "work"]

In [ ]:
import tensorflow as tf

idx = pd.IndexSlice
padding_value = -10000
rewards_list = [
    df_episodes.loc[idx[i, :, :, :, :], idx["reward", "work"]].values.tolist()
    for i in df_episodes.index.levels[0]
]
r_n_t = tf.keras.utils.pad_sequences(
    rewards_list, padding="post", dtype=np.float32, value=padding_value
)
states_list = [
    df_episodes.loc[idx[i, :, :, :, :], idx["state", ["velocity", "thrust", "brake"]]].values.tolist()  # type: ignore
    for i in df_episodes.index.levels[0]
]  # type: ignore
s_n_t = tf.keras.utils.pad_sequences(
    states_list, padding="post", dtype=np.float32, value=padding_value
)
actions_list = [
    df_episodes.loc[idx[i, :, :, :, :], idx["action", torque_table_row_names]].values.tolist()  # type: ignore
    for i in df_episodes.index.levels[0]
]  # type: ignore
a_n_t = tf.keras.utils.pad_sequences(
    actions_list, padding="post", dtype=np.float32, value=padding_value
)
nstates_list = [
    df_episodes.loc[idx[i, :, :, :, :], idx["nstate", ["velocity", "thrust", "brake"]]].values.tolist()  # type: ignore
    for i in df_episodes.index.levels[0]
]  # type: ignore
ns_n_t = tf.keras.utils.pad_sequences(
    nstates_list, padding="post", dtype=np.float32, value=padding_value
)

r_n_t
s_n_t
a_n_t
ns_n_t